In [1]:
import matplotlib
def color_histogram_hsv(im, nbin=100, xmin=0, xmax=255, normalized=True):

    ndim = im.ndim
    bins = np.linspace(xmin, xmax, nbin+1)
    hsv = matplotlib.colors.rgb_to_hsv(im/xmax) * xmax
    imhist, bin_edges = np.histogram(hsv[:,:,0], bins=bins, density=normalized)
    imhist = imhist * np.diff(bin_edges)

    return imhist

In [2]:
# Стратифицированное разбиение на фолды
def skf(a):
    n = len(a)
    b = [0, 0]
    for i in a:
        b[i] += 1
    bb = []
    for i in range(5):
        bb += [[0, 0]]
    for i in range(2):
        for j in range(b[i] % 5):
            bb[j][i] = b[i] // 5 + 1
        for j in range(b[i] % 5, 5):
            bb[j][i] = b[i] // 5;
 
    cur = [0, 0]
    c = []
    for i in range(5):
        c += [[0, 0]]
    ans = []
    for i in range(5):
        ans += [[[], []]]
    for i in range(n):
        if c[cur[a[i]]][a[i]] < bb[cur[a[i]]][a[i]]:
            for j in range(5):
                if (cur[a[i]] == j):
                    ans[j][0].append(i)
                else:
                    ans[j][1].append(i)
            c[cur[a[i]]][a[i]] += 1
            if (c[cur[a[i]]][a[i]] == bb[cur[a[i]]][a[i]]):
                cur[a[i]] += 1;
    return ans

In [3]:
# Трёхмерная гистограмма цветов
def hist3d(img):
    hist = np.zeros((8,8,8))
    tmp = img // 32
    for i in range(32):
        for j in range(32):
            hist[tmp[i,j,0],tmp[i,j,1],tmp[i,j,2]] += 1
    return hist.flatten()

In [4]:
from scipy.ndimage import uniform_filter

# Перевод в серый
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.144])

# Фичи гистограмма направленных градиентов
def hog (im):  

    image = rgb2gray(im)
    # Число направлений
    vect_cnt = 9 
    # Сколько пикселей на ячейку
    cx, cy = (4, 4)

    gx = np.zeros(image.shape)
    gy = np.zeros(image.shape)
    # Градиент по первой координате
    gx[:, :-1] = np.diff(image, n=1, axis=1)
    # Градиент по второй координате
    gy[:-1, :] = np.diff(image, n=1, axis=0)
    # Величина градиента
    val = np.sqrt(gx ** 2 + gy ** 2)
    # Направление градиента
    direction = np.arctan2(gy, (gx + 1e-14)) * (180 / np.pi) + 90 
    

    # Число ячеек по первой координате
    n_cellsx = int(np.floor(image.shape[0] / cx))
    # Число ячеек по второй координате
    n_cellsy = int(np.floor(image.shape[1] / cy))
    hist = np.zeros((n_cellsx, n_cellsy, vect_cnt))
    for i in range(vect_cnt):
        temp_direction = np.where(direction < 180 / vect_cnt * (i + 1), direction, 0)
        temp_direction = np.where(direction >= 180 / vect_cnt * i, temp_direction, 0)

        cond2 = temp_direction > 0
        temp_mag = np.where(cond2, val, 0)
        hist[:,:,i] = uniform_filter(temp_mag, size=(cx, cy))[int(cx/2)::cx, int(cy/2)::cy].T
  
    return hist.ravel()

In [5]:
def compute_direction(x, y, threshold_delta):
    return 1 if x - y > threshold_delta else 0

# 7   0   1
# 6   x   2
# 5   4   3
# Фичи Local binary patterns
def compute_lbp(data, i, j, threshold_delta):
    result = 0
    result +=       compute_direction(data[i, j], data[i-1, j], threshold_delta)
    result += 2   * compute_direction(data[i, j], data[i-1, j+1], threshold_delta)
    result += 4   * compute_direction(data[i, j], data[i, j+1], threshold_delta)
    result += 8   * compute_direction(data[i, j], data[i+1, j+1], threshold_delta)
    result += 16  * compute_direction(data[i, j], data[i+1, j], threshold_delta)
    result += 32  * compute_direction(data[i, j], data[i+1, j-1], threshold_delta)
    result += 64  * compute_direction(data[i, j], data[i, j-1], threshold_delta)
    result += 128 * compute_direction(data[i, j], data[i-1, j-1], threshold_delta)
    return result

In [6]:
import sys
import numpy as np
from scipy.stats import logistic
 
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def logloss(true_label, predicted, eps=1e-15):
    p = np.clip(predicted, eps, 1 - eps)
    if true_label == 1:
        return -np.log(p)
    else:
        return -np.log(1 - p)
    
def log_loss(y_true, y_pred):
    los = 0
    for i in range(len(y_true)):


        los += logloss(y_true[i][1], y_pred[i][1])
            
    return los / len(y_true)    
    
def log_loss1(y_true, y_pred):
    los = 0
    for i in range(len(y_true)):

        los += logloss(y_true[i], y_pred[i])
            
    return los / len(y_true)     

    
def sigmoid(x):
    return logistic.cdf(x)
"""
def softmax(x):
    e = np.exp(x - np.max(x))  # prevent overflow
    if e.ndim == 1:
        return e / np.sum(e, axis=0)
    else:  
        return e / np.array([np.sum(e, axis=1)]).T  # ndim = 2
"""

def softmax(X, copy=True):

    if copy:
        X = np.copy(X)
    max_prob = np.max(X, axis=1).reshape((-1, 1))
    X -= max_prob
    np.exp(X, X)
    sum_prob = np.sum(X, axis=1).reshape((-1, 1))
    X /= sum_prob
    return X

class LogisticRegression_(object):
    def __init__(self, X, y, X_val, y_val):
        self.X = X
        self.y = y
        self.X_val = X_val
        self.y_val = y_val
        self.W = np.zeros((X.shape[1], y.shape[1]))  # initialize W 0
        self.b = np.zeros(y.shape[1])          # initialize bias 0
        self.losses_train = []
        self.losses_valid = []

        # self.params = [self.W, self.b]

    def fit(self, n_epochs=1000, lr=0.0001, L2_reg=0.001):
        for i in range(n_epochs):
            # p_y_given_x = sigmoid(numpy.dot(self.x, self.W) + self.b)
            p_y_given_x = softmax(np.dot(self.X, self.W) + self.b)
            d_y = self.y - p_y_given_x

            self.W += lr * np.dot(self.X.T, d_y) - lr * L2_reg * self.W
            self.b += lr * np.mean(d_y, axis=0)
            #print(softmax(np.dot(self.X, self.W) + self.b))

            #print(softmax(np.dot(self.X_val, self.W) + self.b))
            if i % 500 == 0:
                loss = log_loss(self.y, softmax(np.dot(self.X, self.W) + self.b))
                val_loss = log_loss(self.y_val, softmax(np.dot(self.X_val, self.W) + self.b))
                #loss = self.cross_entropy(val=False)
                #val_loss = self.cross_entropy(val=True)
            
                self.losses_train += [loss]
                self.losses_valid += [val_loss]
                print('Train_loss=', loss,' Val_loss=', val_loss)

    def cross_entropy(self, val=False):
        if val:
            X = self.X_val
            y = self.y_val
        else:
            X = self.X
            y = self.y
            
        # sigmoid_activation = sigmoid(numpy.dot(self.x, self.W) + self.b)
        sigmoid_activation = softmax(np.dot(X, self.W) + self.b)

        cross_entropy = - np.mean(np.sum(y * np.log(sigmoid_activation) + 
                                               (1 - y) * np.log(1 - sigmoid_activation),axis=1))
        return cross_entropy
    
    def predict_proba(self, X):
        # return sigmoid(np.dot(x, self.W) + self.b)
        return softmax(np.dot(X, self.W) + self.b)

    def predict(self, X):
        # return sigmoid(np.dot(x, self.W) + self.b)
        preds = []
        probas = softmax(np.dot(X, self.W) + self.b)
        for i in probas:
            preds += [i == i.max()]
        return np.array(preds).astype(int)
    
    

In [7]:
import os
from scipy import misc
train = os.listdir('train/train')
test = os.listdir('test/test')

train = np.array(train)
test = np.array(test)

tmp = []
for i in train:
    tmp += ['train/train/'+i]
train = np.array(tmp)

tmp = []
for i in test:
    tmp += ['test/test/'+i]
test = np.array(tmp)


classes = []
y = []
for i in train:
    if 'indoor' in i:
        classes += [[0, 1]]
        y += [1]
    else:
        classes += [[1, 0]]
        y += [0]
classes = np.array(classes)
y = np.array(y)

In [79]:
len(test)

2960

### Разбиение на фолды

In [8]:
#y = np.array(y.tolist() * 2)
tmp = skf(y)
    
folds = []    
for i in tmp:
    folds += [[i[1], i[0]]]

### Считывание картинок

In [9]:
x_train = []
for seed in range(8):
    for i in train:
        img = misc.imread(i)
        img = np.rot90(img, k=seed)
        if seed >= 4:
            img = np.fliplr(img)    
        x_train += [img]

x_train = np.array(x_train)


x_test = []
for seed in range(8):
    for i in test:
        img = misc.imread(i)
        img = np.rot90(img, k=seed)
        if seed >= 4:
            img = np.fliplr(img)       
        x_test += [img]
x_test = np.array(x_test)

In [39]:
x_train.shape

(55240, 32, 32, 3)

In [14]:
55240 / 8.

6905.0

In [12]:
len(x_test)

23680

### Цветовые гистограммы

In [10]:
hist_train = []
for i in x_train:
    tmp = []
    for j in range(3):
        tmp += [np.histogram(i[:,:,j], bins=50)[0]]
    hist_train += [tmp]
hist_train = np.array(hist_train).reshape(x_train.shape[0], 150)


hist_test = []
for i in x_test:
    tmp = []
    for j in range(3):
        tmp += [np.histogram(i[:,:,j], bins=50)[0]]
    hist_test += [tmp]
hist_test = np.array(hist_test).reshape(23680, 150)

In [11]:
hist_hsv_train = []
for i in x_train:
    hist_hsv_train += [color_histogram_hsv(i)]
hist_hsv_train = np.array(hist_hsv_train)

hist_hsv_test = []
for i in x_test:
    hist_hsv_test += [color_histogram_hsv(i)]
hist_hsv_test = np.array(hist_hsv_test)

### HOG фичи

In [12]:
hog_train = []
for i in x_train:
    hog_train += [hog(i)]
hog_train = np.array(hog_train)


hog_test = []
for i in x_test:
    hog_test += [hog(i)]
hog_test = np.array(hog_test)

### LBP фичи

In [13]:
lbp_train = []
for t in x_train:
    image = t
    image = rgb2gray(image)

    lbps_histdata = []
    mean_delta = 0

    for i in range(1, image.shape[1]-1):
        for j in range(1, image.shape[0]-1):
            mean_delta += (abs(image[i, j] - image[i, j-1]) + abs(image[i, j] - image[i-1, j-1]) +
                abs(image[i, j] - image[i-1, j]) + abs(image[i, j] - image[i-1, j+1]) )/4.0
    mean_delta /= (image.shape[1]-2) * (image.shape[0]-2) 
    mean_delta *= 1.5 

    for i in range(0, image.shape[1]):
        for j in range(0, image.shape[0]):
            if i != 0 and j != 0 and i != image.shape[1]-1 and j != image.shape[0]-1:
                tmp = compute_lbp(image, i, j, mean_delta)
                if tmp != 0:
                    lbps_histdata.append(tmp)
    hist, bins = np.histogram(lbps_histdata, bins=254, normed=True)
                 
    lbp_train += [hist]
lbp_train = np.array(lbp_train)




lbp_test = []
for t in x_test:
    image = t
    image = rgb2gray(image)

    lbps_histdata = []
    mean_delta = 0

    for i in range(1, image.shape[1]-1):
        for j in range(1, image.shape[0]-1):
            mean_delta += (abs(image[i, j] - image[i, j-1]) + abs(image[i, j] - image[i-1, j-1]) +
                abs(image[i, j] - image[i-1, j]) + abs(image[i, j] - image[i-1, j+1]) )/4.0
    mean_delta /= (image.shape[1]-2) * (image.shape[0]-2) 
    mean_delta *= 1.5 

    for i in range(0, image.shape[1]):
        for j in range(0, image.shape[0]):
            if i != 0 and j != 0 and i != image.shape[1]-1 and j != image.shape[0]-1:
                tmp = compute_lbp(image, i, j, mean_delta)
                if tmp != 0:
                    lbps_histdata.append(tmp)
    hist, bins = np.histogram(lbps_histdata, bins=254, normed=True)
                 
    lbp_test += [hist]
lbp_test = np.array(lbp_test)


### Трёхмерные гистограммы цвета

In [14]:
hist3d_train = []
for i in x_train:
    hist = hist3d(i)
    hist3d_train += [hist]
hist3d_train = np.array(hist3d_train)


hist3d_test = []
for i in x_test:
    hist = hist3d(i)
    hist3d_test += [hist]
hist3d_test = np.array(hist3d_test)

### Фичи - статистики

In [15]:
ms_train = []
for i in x_train:
    tmp = []
    for j in range(3):
        tmp += [i[:,:,j].mean()]
        tmp += [i[:,:,j].std()]
        tmp += [np.median(i[:,:,j])]
    ms_train += [tmp]
ms_train = np.array(ms_train)


ms_test = []
for i in x_test:
    tmp = []
    for j in range(3):
        tmp += [i[:,:,j].mean()]
        tmp += [i[:,:,j].std()]
        tmp += [np.median(i[:,:,j])]
    ms_test += [tmp]
ms_test = np.array(ms_test)

In [218]:
#x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2]*x_train.shape[3])
#x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2]*x_test.shape[3])

### Нормализация  и склейка фичей

In [16]:
hog_test = hog_test / hog_train.max() * 1.
hog_train = hog_train / hog_train.max() * 1.

hist3d_test = hist3d_test / hist3d_train.max() * 1.
hist3d_train = hist3d_train / hist3d_train.max() * 1.

hist_test = hist_test / hist_train.max() * 1.
hist_train = hist_train / hist_train.max() * 1.

ms_test = ms_test / 255.
ms_train = ms_train / 255.

In [17]:
from matplotlib.mlab import PCA

tmp = PCA(np.vstack([x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2]*x_train.shape[3]), x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2]*x_test.shape[3])]))
tmp = tmp.Y[:, :200]

In [25]:
x_train.shape

(55240, 32, 32, 3)

In [26]:
x_test.shape

(2960, 32, 32, 3)

In [18]:
pca_train = tmp[:55240]
pca_test = tmp[55240:]

In [28]:
len(tmp[55240:])

2960

In [19]:
pca_test = pca_test / pca_train.max() * 1.
pca_train = pca_train / pca_train.max() * 1.

In [31]:
pca_test.shape

(2960, 200)

In [20]:
tr = np.hstack([hist_train, hog_train, lbp_train, hist3d_train, pca_train])
te = np.hstack([hist_test, hog_test, lbp_test, hist3d_test, pca_test])

In [51]:
tr.shape

(55240, 1692)

## Обучение

In [21]:
pred_train = np.zeros(len(tr))
est = []
f_ll,f_rmse = [],[]

for itr, ite in folds:
    ypred = []
    tmp_tr = tr[itr]
    for part in range(1,8):   
        tmp_tr = np.vstack([tmp_tr, tr[6905*part + np.array(itr)]])
    tmp_classes = np.array(classes[itr].tolist()*8)
    classifier = LogisticRegression_(tmp_tr, tmp_classes, tr[ite], classes[ite])
    classifier.fit(n_epochs=2500, lr=0.0005, L2_reg=0.0001)
    
    ypred = classifier.predict_proba(tr[ite])[:, 1]
    for part in range(1, 8):
        ypred += classifier.predict_proba(tr[6905*part + np.array(ite)])[:, 1]

    
    pred_train[ite] = ypred / 8.

    print(log_loss1(y[ite], pred_train[ite]))
    print(rmse(y[ite], pred_train[ite]))
    
    f_ll.append(log_loss1(y[ite], pred_train[ite]))
    f_rmse.append(rmse(y[ite], pred_train[ite]))

    est.append(classifier)
    
print('logreg: {:.3f} +- {:.3f}'.format(np.mean(f_ll), np.std(f_ll)))
print('rmse: {:.3f} +- {:.3f}'.format(np.mean(f_rmse), np.std(f_rmse)))

Train_loss= 0.913184970785  Val_loss= 0.912074937933
Train_loss= 0.150315958815  Val_loss= 0.194097580043
Train_loss= 0.138417213279  Val_loss= 0.187906615238
Train_loss= 0.132222218223  Val_loss= 0.186064013652
Train_loss= 0.128091815106  Val_loss= 0.185659447835
0.173230693009
0.218392669878
Train_loss= 0.915223404768  Val_loss= 0.919865740666
Train_loss= 0.156763318372  Val_loss= 0.165326717076
Train_loss= 0.144722965838  Val_loss= 0.157404527964
Train_loss= 0.138381560772  Val_loss= 0.15487001789
Train_loss= 0.134138290492  Val_loss= 0.153960900617
0.142587653742
0.200227637994
Train_loss= 0.914438181716  Val_loss= 0.907615697138
Train_loss= 0.151647664075  Val_loss= 0.185465283039
Train_loss= 0.139700672325  Val_loss= 0.180127668733
Train_loss= 0.133507378516  Val_loss= 0.178352286527
Train_loss= 0.129419882353  Val_loss= 0.177667101418
0.167165690236
0.217179976161
Train_loss= 0.922098501607  Val_loss= 0.930333685026
Train_loss= 0.160041253969  Val_loss= 0.150762141662
Train_loss

In [22]:
import pandas as pd
pd.DataFrame(pred_train).to_csv('preds/3_train.csv', index=False, header=None)

In [24]:
# Усреднение 
res = est[0].predict_proba(te[:2960])
for part in range(1,8):
    res += est[0].predict_proba(te[2960 * part:2960 * (part+1)])

for i in range(1,5):
    for part in range(8):
        res += est[i].predict_proba(te[2960 * part:2960 * (part+1)])

res = res / 40.0

In [25]:
tmp = pd.DataFrame(test)
tmp.columns = ['id']
tmp['id'] = tmp['id'].apply(lambda x: int(x.split('_')[1][:-4]))
tmp['res'] = res[:,1]
tmp.to_csv('preds/3.csv', index=False)

In [26]:
tmp

,id,res
0,223,0.042007
1,1878,0.987742
2,409,0.828963
3,1993,0.742050
4,783,0.981607
5,900,0.275407
6,2508,0.976372
7,1848,0.999862
8,1285,0.987167
9,2889,0.118860
